In [19]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [20]:
## Loading dataset
df_names = pd.read_csv("name_gender_dataset.csv")

In [21]:
## checking Dataset
df_names.head(10)
#print(df_names[df_names['Count'] == 10])

,Name,Gender,Count,Probability
0,James,M,5304407,0.014517
1,John,M,5260831,0.014398
2,Robert,M,4970386,0.013603
3,Michael,M,4579950,0.012534
4,William,M,4226608,0.011567
5,Mary,F,4169663,0.011411
6,David,M,3787547,0.010366
7,Joseph,M,2695970,0.007378
8,Richard,M,2638187,0.007220
9,Charles,M,2433540,0.006660


In [22]:
## Checking dimensions of the dataframe
print(df_names.shape)

(147269, 4)


In [23]:
df_names['length'] = df_names['Name'].str.len()
df_names.sort_values('length', ascending=True, inplace=True)
df_names.head(100)

,Name,Gender,Count,Probability,length
111926,I,M,3,8.210220e-09,1
128463,R,F,1,2.736740e-09,1
111931,J,M,3,8.210220e-09,1
111009,M,M,4,1.094700e-08,1
114150,A,M,2,5.473480e-09,1
...,...,...,...,...,...
72012,No,F,18,4.926130e-08,2
59004,Vy,M,37,1.012590e-07,2
67969,Ye,M,23,6.294500e-08,2
69165,In,F,21,5.747160e-08,2


In [24]:
df_names = df_names.sort_values(by='length', ascending=False)
df_names


,Name,Gender,Count,Probability,length
135869,Carmelo-Antonio-Francesco,M,1,2.736740e-09,25
129608,Sandrine-Marie-Madeleine,F,1,2.736740e-09,24
142268,Nathan-Conway-Barrington,M,1,2.736740e-09,24
128565,Rajeswary@Rajo@Chanchal,F,1,2.736740e-09,23
141674,Merlin-Siegfried-Daniel,M,1,2.736740e-09,23
...,...,...,...,...,...
136352,D,M,1,2.736740e-09,1
127712,O,F,1,2.736740e-09,1
119136,D,F,1,2.736740e-09,1
95606,P,M,6,1.642040e-08,1


In [25]:
## We founnd probability is useless here so we drop the column
df_names = df_names.drop('Probability', axis=1)
df_names

,Name,Gender,Count,length
135869,Carmelo-Antonio-Francesco,M,1,25
129608,Sandrine-Marie-Madeleine,F,1,24
142268,Nathan-Conway-Barrington,M,1,24
128565,Rajeswary@Rajo@Chanchal,F,1,23
141674,Merlin-Siegfried-Daniel,M,1,23
...,...,...,...,...
136352,D,M,1,1
127712,O,F,1,1
119136,D,F,1,1
95606,P,M,6,1


In [26]:
## Changing the gender to numerical classifiers

df_names['Gender'] = df_names['Gender'].replace({'M': 1, 'F': 0})
df_names.head(100)

,Name,Gender,Count,length
135869,Carmelo-Antonio-Francesco,1,1,25
129608,Sandrine-Marie-Madeleine,0,1,24
142268,Nathan-Conway-Barrington,1,1,24
128565,Rajeswary@Rajo@Chanchal,0,1,23
141674,Merlin-Siegfried-Daniel,1,1,23
...,...,...,...,...
130126,Shakeisha-Kerryann,0,1,18
139260,Jefferson-Michael,1,1,17
118224,Carmena-Josephine,0,1,17
130140,"Shakuntala@Pooja,",0,1,17


In [27]:
## Remove names with one letter and bigger than 15
df_names.drop(df_names[df_names['length'] > 15].index, inplace = True)
df_names.drop(df_names[df_names['length'] == 1].index, inplace = True)
df_names.head(10)

,Name,Gender,Count,length
119244,Danielle-Giulia,0,1,15
138980,Jakob-Alexander,1,1,15
131050,Sonnja-Gabriele,0,1,15
133596,Yirgjhilya-Mary,0,1,15
135383,Bertlee-Brendon,1,1,15
120377,Erica-Elisabeta,0,1,15
118459,Chantal-Jessica,0,1,15
119245,Danielle-Nicole,0,1,15
107598,Jordanchristoph,1,5,15
118359,Celeste-Monique,0,1,15


In [28]:
# Group by all columns and calculate size (count) for each group
df_names.drop(df_names[df_names['Count'] <= 5].index, inplace = True)
df_names.sort_values(by="length", ascending=True).head(10)


,Name,Gender,Count,length
41630,Zi,0,98,2
22037,Ab,1,362,2
38069,Kj,1,122,2
69454,Si,0,21,2
88508,Un,1,8,2
12735,Vu,1,876,2
24258,Ly,0,304,2
27176,Na,0,246,2
69386,Ra,0,21,2
80560,Te,1,12,2


In [29]:
df_names = df_names[~df_names['Name'].str.contains('-')]
df_names = df_names[~df_names['Name'].str.contains('@')]
df_names

,Name,Gender,Count,length
83145,Matthewalexande,1,11,15
68650,Christopherjose,1,22,15
82639,Christopherryan,1,11,15
94448,Christopheranth,1,6,15
68362,Mariadelrosario,0,22,15
...,...,...,...,...
67038,Oc,1,24,2
62215,Qi,1,31,2
64796,Ke,1,27,2
70415,Tj,0,20,2


In [30]:
## Null check
nan_check = df_names.isna().sum()
nan_check

Name      0
Gender    0
Count     0
length    0
dtype: int64

In [31]:
import pandas as pd

# Sample data (replace with your actual DataFrame)
data = {'name': ['Alice', 'Bob', 'Charlie', 'David', 'Emily']}
df = pd.DataFrame(data)

# New column to store last three letters
df['last_three_letters'] = df['name'].str[-3:]

# Print the DataFrame with the new column
print(df)


      name last_three_letters
0    Alice                ice
1      Bob                Bob
2  Charlie                lie
3    David                vid
4    Emily                ily


In [32]:
## names to lowercase to dont have problems with letters 
df_names['Name'] = df_names['Name'].apply(lambda x: x.upper() if isinstance(x, str) else x)
df_names.head(15)

,Name,Gender,Count,length
83145,MATTHEWALEXANDE,1,11,15
68650,CHRISTOPHERJOSE,1,22,15
82639,CHRISTOPHERRYAN,1,11,15
94448,CHRISTOPHERANTH,1,6,15
68362,MARIADELROSARIO,0,22,15
68649,CHRISTIANJOSEPH,1,22,15
89951,CHRISTIANDANIEL,1,7,15
37068,FRANCISCOJAVIER,1,130,15
82638,CHRISTIANMICHAE,1,11,15
79992,CHRISTIANANTHON,1,12,15


In [33]:
# df_names['l3_letters'] = df_names['Name'].str[-3:]
# df_names['l2_letters'] = df_names['Name'].str[-2:]
# df_names['l1_letters'] = df_names['Name'].str[-1:]
df_names.sort_values(by="length", ascending=False).head(20)

## print(df_names.shape)

,Name,Gender,Count,length
83145,MATTHEWALEXANDE,1,11,15
50449,CHRISTOPHERMICH,1,59,15
68650,CHRISTOPHERJOSE,1,22,15
87663,ASHLEYELIZABETH,0,8,15
86869,SEANCHRISTOPHER,1,10,15
95392,MARKCHRISTOPHER,1,6,15
66208,JONATHANMICHAEL,1,25,15
38594,CHRISTOPHERJOHN,1,118,15
94763,GABRIELALEXANDE,1,6,15
59182,MARIADELOSANGEL,0,36,15


In [34]:
# Function to convert string to ASCII values and pad with zeros if length is less than 5
def string_to_ascii_padded(s):
    ascii_values = [(ord(c)-64) for c in s]
    while len(ascii_values) < 15:
        ascii_values.append(0)
    return ascii_values

# Apply the function to the 'names' column
df_names['encoded_names'] = df_names['Name'].apply(string_to_ascii_padded)
## df_names = df_names.drop('names_ascii', axis=1)

df_names['name_ascii'] = [','.join(map(str, l)) for l in df_names['encoded_names']]

df_names = df_names.drop('encoded_names', axis=1)

df_names.sort_index().head(20)

,Name,Gender,Count,length,name_ascii
0,JAMES,1,5304407,5,"10,1,13,5,19,0,0,0,0,0,0,0,0,0,0"
1,JOHN,1,5260831,4,"10,15,8,14,0,0,0,0,0,0,0,0,0,0,0"
2,ROBERT,1,4970386,6,"18,15,2,5,18,20,0,0,0,0,0,0,0,0,0"
3,MICHAEL,1,4579950,7,"13,9,3,8,1,5,12,0,0,0,0,0,0,0,0"
4,WILLIAM,1,4226608,7,"23,9,12,12,9,1,13,0,0,0,0,0,0,0,0"
5,MARY,0,4169663,4,"13,1,18,25,0,0,0,0,0,0,0,0,0,0,0"
6,DAVID,1,3787547,5,"4,1,22,9,4,0,0,0,0,0,0,0,0,0,0"
7,JOSEPH,1,2695970,6,"10,15,19,5,16,8,0,0,0,0,0,0,0,0,0"
8,RICHARD,1,2638187,7,"18,9,3,8,1,18,4,0,0,0,0,0,0,0,0"
9,CHARLES,1,2433540,7,"3,8,1,18,12,5,19,0,0,0,0,0,0,0,0"


In [50]:
# Split the 'encoded_names' column into separate columns
df_pre = df_names.drop(["Name", "Count", "length"], axis=1)
df_pre


,Gender,name_ascii
83145,1,"13,1,20,20,8,5,23,1,12,5,24,1,14,4,5"
68650,1,"3,8,18,9,19,20,15,16,8,5,18,10,15,19,5"
82639,1,"3,8,18,9,19,20,15,16,8,5,18,18,25,1,14"
94448,1,"3,8,18,9,19,20,15,16,8,5,18,1,14,20,8"
68362,0,"13,1,18,9,1,4,5,12,18,15,19,1,18,9,15"
...,...,...
67038,1,"15,3,0,0,0,0,0,0,0,0,0,0,0,0,0"
62215,1,"17,9,0,0,0,0,0,0,0,0,0,0,0,0,0"
64796,1,"11,5,0,0,0,0,0,0,0,0,0,0,0,0,0"
70415,0,"20,10,0,0,0,0,0,0,0,0,0,0,0,0,0"


In [56]:
name_ascii_split = df_names['name_ascii'].str.split(',', expand=True)

# Concatenate the new DataFrame with the 'Gender' column
result_df= pd.concat([df_names['Gender'], name_ascii_split], axis=1)

result_df = result_df.astype(int)
# result_df = result_df[["Gender","0","1","2","3","4","5","6","7","8","9","10","11","12","13","14"]]

cols = result_df.columns.tolist()
cols.append(cols.pop(cols.index('Gender')))
df_pro = result_df[cols]

df_pro

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Gender
83145,13,1,20,20,8,5,23,1,12,5,24,1,14,4,5,1
68650,3,8,18,9,19,20,15,16,8,5,18,10,15,19,5,1
82639,3,8,18,9,19,20,15,16,8,5,18,18,25,1,14,1
94448,3,8,18,9,19,20,15,16,8,5,18,1,14,20,8,1
68362,13,1,18,9,1,4,5,12,18,15,19,1,18,9,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67038,15,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1
62215,17,9,0,0,0,0,0,0,0,0,0,0,0,0,0,1
64796,11,5,0,0,0,0,0,0,0,0,0,0,0,0,0,1
70415,20,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0
